## Imports

In [1]:
# Basic imports
import pandas as pd
import numpy as np

# Warnings
import warnings 
warnings.simplefilter("ignore")

In [2]:
# Plot
from IPython import display
import seaborn as sns

import matplotlib
import matplotlib.pylab as plt
from jupyterthemes import jtplot

jtplot.style('gruvboxd')
matplotlib.use('nbagg')

# Data Reading

In [3]:
from catboost.datasets import titanic

# Data Reading
df_train, df_test = titanic()
df_train.set_index('PassengerId', inplace=True)
df_test.set_index('PassengerId', inplace=True)

# Split X_train, y_train
target = 'Survived'
features = df_test.columns

y_train = df_train[target]
df_train = df_train[features]

df_train.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Data Preparation

In [4]:
from robusta.preprocessing.category import *
from robusta.preprocessing.numeric import *
from robusta.preprocessing import *
from robusta.compose import *

nums = ['Age', 'Fare', 'SibSp', 'Parch']
cats = ['Pclass', 'Sex', 'Embarked']

data_prep = FeatureUnion([
        ("numeric", make_pipeline(
            ColumnSelector(nums),
            Imputer(strategy="median"),
            GaussRank(),
            #ColumnRenamer(prefix='gr_'),
        )),
        ("category", make_pipeline(
            ColumnSelector(cats),
            LabelEncoder(dtype='category'),
            #ColumnRenamer(prefix='le_'),
        )),
])

X_train = data_prep.fit_transform(df_train)
X_test = data_prep.transform(df_test)

X_train.head()

,Age,Fare,SibSp,Parch,Pclass,Sex,Embarked
PassengerId,,,,,,,
1,-0.488867,-0.968076,0.600843,-0.214091,2,1,2
2,0.584412,0.853865,0.600843,-0.214091,0,0,0
3,-0.275357,-0.454031,-0.290348,-0.214091,2,0,2
4,0.470091,0.703973,0.600843,-0.214091,0,0,2
5,0.470091,-0.379298,-0.290348,-0.214091,2,1,2


# Fold Preparation

In [5]:
%%time
from lightgbm import LGBMClassifier

model = LGBMClassifier()
model.fit(X_train, y_train)

CPU times: user 583 ms, sys: 53.6 ms, total: 637 ms
Wall time: 115 ms


/Users/pechyonkinb/anaconda/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [6]:
#fold_prep = FeatureUnion([
#    ('category', make_pipeline(
#        TypeSelector(['category', 'object']), 
#        TargetEncoderCV(cv=te_cv, smoothing=1))),
#    ('numeric', TypeSelector(np.number))
#])
fold_prep = ColumnTransformer([
    ('cats', TargetEncoderCV(cv=4).set_params(encoder__smoothing=2.0), cats),
    ('nums', Identity(), nums)
])
    
F_train = fold_prep.fit_transform(X_train, y_train)

F_train.sample(5, random_state=555)

,Embarked,Pclass,Sex,Age,Fare,SibSp,Parch
PassengerId,,,,,,,
876,0.539062,0.250000,0.739669,-0.944113,-1.132927,-0.290348,-0.214091
595,0.336082,0.442857,0.177986,0.560940,0.277085,0.600843,-0.214091
133,0.334711,0.217514,0.741379,0.888832,0.016949,0.600843,-0.214091
233,0.341615,0.500000,0.201357,1.332614,-0.027920,-0.290348,-0.214091
506,0.532787,0.629870,0.177986,-0.757656,1.132927,0.600843,-0.214091


In [7]:
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import *

estimator = make_pipeline(fold_prep, model)

cv = 5
scoring = 'neg_log_loss'
#scoring = 'accuracy'

scores = cross_val_score(estimator, X_train, y_train, cv=cv, scoring=scoring)

print('{:.4f} ± {:.4f}'.format(np.mean(scores), np.std(scores)))

-0.4780 ± 0.0828


In [13]:
from sklearn.model_selection import GridSearchCV

param = 'columntransformer__cats__encoder__smoothing'
param_space = [1, 5, 10, 50, 100, 150, 200, 250, 300, 500]
space = {param: param_space}

gs = GridSearchCV(model, space, cv=cv, scoring=scoring)
gs.fit(X_train, y_train)

/Users/pechyonkinb/anaconda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('columntransformer', ColumnTransformer(remainder='drop',
         transformer_list=[('cats', EncoderCV(cv=4,
     encoder=TargetEncoder(cols=None, drop_invariant=False, handle_missing='value',
       handle_unknown='value', min_samples_leaf=1, return_df=True,
       smoothing=2.0, verbose=0)...0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'columntransformer__cats__encoder__smoothing': [1, 5, 10, 50, 100, 150, 200, 250, 300, 500]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_log_loss', verbose=0)

In [14]:
pd.DataFrame(gs.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_columntransformer__cats__encoder__smoothing,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
0,0.693772,0.116457,0.296922,0.043595,1,{'columntransformer__cats__encoder__smoothing'...,-0.492346,-0.629424,-0.405333,-0.474433,...,-0.480619,0.083069,8,-0.135267,-0.11773,-0.132926,-0.133479,-0.138392,-0.131559,0.007173
1,0.685597,0.023581,0.292922,0.048246,5,{'columntransformer__cats__encoder__smoothing'...,-0.480666,-0.629424,-0.405333,-0.474433,...,-0.478273,0.082870,5,-0.135268,-0.11773,-0.132926,-0.133479,-0.138392,-0.131559,0.007173
2,0.638818,0.005563,0.270583,0.037635,10,{'columntransformer__cats__encoder__smoothing'...,-0.480666,-0.629424,-0.405333,-0.474433,...,-0.478273,0.082870,5,-0.135268,-0.11773,-0.132926,-0.133479,-0.138392,-0.131559,0.007173
3,0.642502,0.006213,0.308049,0.047900,50,{'columntransformer__cats__encoder__smoothing'...,-0.517186,-0.630715,-0.386426,-0.474433,...,-0.482092,0.088640,10,-0.137735,-0.11773,-0.139078,-0.133479,-0.138392,-0.133283,0.008018
4,0.687537,0.047652,0.269479,0.037923,100,{'columntransformer__cats__encoder__smoothing'...,-0.507194,-0.630715,-0.386426,-0.479731,...,-0.481143,0.087890,9,-0.137735,-0.11773,-0.139078,-0.133479,-0.138392,-0.133283,0.008018
5,0.646493,0.015952,0.289720,0.049411,150,{'columntransformer__cats__encoder__smoothing'...,-0.507194,-0.630715,-0.386426,-0.474433,...,-0.480085,0.087933,7,-0.137735,-0.11773,-0.139078,-0.133479,-0.138392,-0.133283,0.008018
6,0.642030,0.013646,0.247805,0.001128,200,{'columntransformer__cats__encoder__smoothing'...,-0.481076,-0.630715,-0.388877,-0.474433,...,-0.475228,0.086525,1,-0.136758,-0.11773,-0.137882,-0.133479,-0.137879,-0.132745,0.007679
7,0.639061,0.008774,0.334339,0.042818,250,{'columntransformer__cats__encoder__smoothing'...,-0.481076,-0.630715,-0.388877,-0.474433,...,-0.475228,0.086525,1,-0.136758,-0.11773,-0.137882,-0.133479,-0.137879,-0.132745,0.007679
8,0.640825,0.013611,0.292416,0.049679,300,{'columntransformer__cats__encoder__smoothing'...,-0.481076,-0.630715,-0.388877,-0.474433,...,-0.475228,0.086525,1,-0.136758,-0.11773,-0.137882,-0.133479,-0.137879,-0.132745,0.007679
9,0.637583,0.007654,0.354907,0.002995,500,{'columntransformer__cats__encoder__smoothing'...,-0.483707,-0.630715,-0.388877,-0.474433,...,-0.475756,0.086567,4,-0.136758,-0.11773,-0.137882,-0.133479,-0.137879,-0.132745,0.007679


In [16]:
opt_model = gs.best_estimator_

Pipeline(memory=None,
     steps=[('columntransformer', ColumnTransformer(remainder='drop',
         transformer_list=[('cats', EncoderCV(cv=4,
     encoder=TargetEncoder(cols=None, drop_invariant=False, handle_missing='value',
       handle_unknown='value', min_samples_leaf=1, return_df=True,
       smoothing=200, verbose=0)...0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0))])